<a href="https://colab.research.google.com/github/EveMurta/Assistente_Virtual_Comando_Voz_Empresa/blob/main/assistente_dados_empresa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ⛺ Variáveis de ambiente

In [ ]:
language = "pt"

### 🎤 **Gravador de áudio**🦻

In [ ]:
#Instalando bibliotecas
!pip -q install pydub

## ✨ Execute:

In [ ]:
# Ref: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
# Importações
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment

#Código em JS para captar gravador no navegador
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  #Executa o JS
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec*1000))
  #Decodifica o audio em base 64
  audio = b64decode(js_result.split(',')[1])
  #Variável que contém o audio
  file_name = "prompt.wav"
  #abre o arquivo para escrever
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print('Ouvindo...\n')
record_file = record()
display(Audio(record_file, autoplay=False))

Ouvindo...



<IPython.core.display.Javascript object>

## 🗣 Transcrição de áudio com Whisper 🧠

In [ ]:
#Instala a biblioteca
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.1 MB/s eta 0:00:00


## ✨ Execute:

In [ ]:
import whisper
# Biblioteca de otimização de resposta
import torch

# Modelos tiny:1GB base:1GB small:2GB medium:5GB large:10GB
model = whisper.load_model("small", device="cuda" if torch.cuda.is_available() else "cpu")

# Transcrição do áudio
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

# Incrementando a transcrição com o prompt para que o chat responda
#formatted_transcription = f"Corrija os erros do texto a seguir respondendo apenas o output(sem a palavra output e sem o input). [Input: ual a ora Output: qual a hora?] Você perguntou{transcription}?"

# Exibindo o resultado formatado
#print(formatted_transcription)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


 Qual foi o maior faturamento deste ano?


## 🎲 Criando o DataFrame 🎲

In [ ]:
import pandas as pd
from transformers import pipeline

# Criando um DataFrame com perguntas e respostas relacionadas aos temas fornecidos
data = {
    "Pergunta": [
        "Qual é o forecast de vendas para o próximo trimestre?",
        "Quantos computadores estão previstos para produção este mês?",
        "Qual foi o faturamento total no último trimestre?",
        "Qual o valor do inventário atual de matéria-prima?",
        "Quantas unidades foram vendidas no mês passado?",
        "Qual é o forecast de demanda para o próximo ano?",
        "Quais são os principais fatores que afetam o forecast de produção?",
        "Quantos computadores foram produzidos na última ordem de produção?",
        "Como foi o crescimento do faturamento em comparação ao ano anterior?",
        "Qual o nível mínimo de inventário de matéria-prima necessário para produção contínua?",
        "Qual foi a quantidade de computadores vendidos no último trimestre?",
        "Como o forecast influencia a produção de computadores?",
        "Qual a diferença entre o forecast inicial e a produção real?",
        "Quais são as tendências de faturamento para o próximo semestre?",
        "Qual o nível de inventário de matéria-prima projetado para o próximo mês?",
        "Qual o forecast de produção para o próximo mês?",
        "Quantas ordens de produção foram emitidas este ano?",
        "Como o inventário de matéria-prima afeta a produção?",
        "O que acontece se o forecast for subestimado?",
        "Qual a diferença entre forecast e pedido real de produção?",
        "Qual foi o maior faturamento mensal deste ano?",
        "Como o forecast de vendas influencia o planejamento de produção?",
        "Qual o nível de inventário de matéria-prima em excesso aceitável?",
        "Como o faturamento está relacionado à quantidade vendida?",
        "O forecast foi revisado nos últimos três meses?",
        "Qual a previsão de inventário de matéria-prima para o próximo trimestre?",
        "Qual foi o impacto de uma baixa no inventário de matéria-prima na produção?",
        "Quantos computadores foram vendidos no ano passado?",
        "Como é feita a previsão de demanda para o forecast?",
        "O faturamento deste trimestre superou o forecast?",
        "Qual é o forecast de vendas para o próximo trimestre?",
        "Quantos computadores estão previstos para produção este mês?",
        "Qual foi o faturamento total no último trimestre?",
        "Qual o valor do inventário atual de matéria-prima?",
        "Quantas unidades de computadores foram vendidas no mês passado?",
        "Qual é o forecast de demanda para o próximo ano?",
        "Quais são os principais fatores que afetam o forecast de produção?",
        "Quantos computadores foram produzidos na última ordem de produção?",
        "Como foi o crescimento do faturamento em comparação ao ano anterior?",
        "Qual o nível mínimo de inventário de matéria-prima necessário para produção contínua?",
        "Qual foi a quantidade de computadores vendidos no último trimestre?",
        "Como o forecast influencia a produção de computadores?",
        "Qual a diferença entre o forecast inicial e a produção real?",
        "Quais são as tendências de faturamento para o próximo semestre?",
        "Qual o nível de inventário de matéria-prima projetado para o próximo mês?",
        "Qual o forecast de produção para o próximo mês?",
        "Quantas ordens de produção foram emitidas este ano?",
        "Como o inventário de matéria-prima afeta a produção?",
        "O que acontece se o forecast for subestimado?",
        "Qual a diferença entre forecast e pedido real de produção?",
        "Qual foi o maior faturamento mensal deste ano?",
        "Como o forecast de vendas influencia o planejamento de produção?",
        "Qual o nível de inventário de matéria-prima em excesso aceitável?",
        "Como o faturamento está relacionado à quantidade vendida?",
        "O forecast foi revisado nos últimos três meses?",
        "Qual a previsão de inventário de matéria-prima para o próximo trimestre?",
        "Qual foi o impacto de uma baixa no inventário de matéria-prima na produção?",
        "Quantos computadores foram vendidos no ano passado?",
        "Como é feita a previsão de demanda para o forecast?",
        "O faturamento deste trimestre superou o forecast?",
        "Qual foi o forecast de vendas do mês passado?",
        "Quantas unidades foram produzidas no mês passado?",
        "Qual foi o desempenho das vendas no mês passado comparado ao forecast?",
        "Quantas ordens de produção foram concluídas no mês passado?",
        "Como o inventário de matéria-prima foi utilizado no mês passado?",
        "Qual é a previsão de crescimento de vendas para o próximo trimestre?",
        "Como a produção planejada para o próximo mês vai se alinhar ao forecast?",
        "Qual o impacto previsto de uma alta na demanda no próximo trimestre?",
        "O forecast de produção do próximo mês já foi ajustado?",
        "Qual é a expectativa de crescimento para o faturamento no próximo ano?",
        "Como o desempenho de vendas do mês passado influenciou a revisão do forecast?",
        "O inventário de matéria-prima está adequado para o forecast do próximo trimestre?"
    ],
    "Resposta": [
        "O forecast de vendas para o próximo trimestre é de 5000 unidades.",
        "Estão previstos 3000 computadores para produção este mês.",
        "O faturamento total no último trimestre foi de R$ 5 milhões.",
        "O inventário atual de matéria-prima está avaliado em R$ 2 milhões.",
        "Foram vendidas 2500 unidades no mês passado.",
        "O forecast de demanda para o próximo ano é de 20000 unidades.",
        "Os principais fatores incluem demanda do mercado, tendências econômicas e capacidade de produção.",
        "Foram produzidos 3200 computadores na última ordem de produção.",
        "O crescimento do faturamento foi de 15% em comparação ao ano anterior.",
        "O nível mínimo de inventário é de 500 toneladas de matéria-prima.",
        "No último trimestre, foram vendidos 2700 computadores.",
        "O forecast determina quantos computadores precisam ser produzidos para atender à demanda.",
        "A produção real foi 10% maior que o forecast inicial.",
        "As tendências de faturamento mostram um crescimento de 8% no próximo semestre.",
        "O inventário projetado para o próximo mês é de 1800 toneladas de matéria-prima.",
        "O forecast de produção para o próximo mês é de 4000 computadores.",
        "Foram emitidas 25 ordens de produção este ano.",
        "O inventário de matéria-prima garante a continuidade da produção e evita paradas inesperadas.",
        "Se o forecast for subestimado, a demanda pode não ser atendida, causando perda de vendas e clientes insatisfeitos.",
        "O forecast é uma estimativa da demanda futura, enquanto o pedido real reflete a produção já confirmada.",
        "O maior faturamento mensal deste ano foi de R$ 6 milhões, registrado em julho.",
        "O forecast de vendas orienta o planejamento da produção para garantir que a oferta atenda à demanda esperada.",
        "Um excesso de até 10% no inventário de matéria-prima é considerado aceitável para evitar rupturas na produção.",
        "O faturamento é diretamente proporcional à quantidade vendida, sendo impactado também pelo preço de venda por unidade.",
        "Sim, o forecast foi revisado duas vezes nos últimos três meses devido a mudanças na demanda do mercado.",
        "A previsão de inventário de matéria-prima para o próximo trimestre é de 2500 toneladas.",
        "A baixa no inventário de matéria-prima resultou em uma redução de 15% na produção no último mês.",
        "No ano passado, foram vendidos 18.000 computadores.",
        "A previsão de demanda é feita com base em dados históricos de vendas, análise de mercado e tendências econômicas.",
        "Sim, o faturamento deste trimestre superou o forecast em 12%.",
        "O forecast de vendas para o próximo trimestre é de 5000 unidades.",
        "Estão previstos 3000 computadores para produção este mês.",
        "O faturamento total no último trimestre foi de R$ 5 milhões.",
        "O inventário atual de matéria-prima está avaliado em R$ 2 milhões.",
        "Foram vendidas 2500 unidades de computadores no mês passado.",
        "O forecast de demanda para o próximo ano é de 20000 unidades.",
        "Os principais fatores incluem demanda do mercado, tendências econômicas e capacidade de produção.",
        "Foram produzidos 3200 computadores na última ordem de produção.",
        "O crescimento do faturamento foi de 15% em comparação ao ano anterior.",
        "O nível mínimo de inventário é de 500 toneladas de matéria-prima.",
        "No último trimestre, foram vendidos 2700 computadores.",
        "O forecast determina quantos computadores precisam ser produzidos para atender à demanda.",
        "A produção real foi 10% maior que o forecast inicial.",
        "As tendências de faturamento mostram um crescimento de 8% no próximo semestre.",
        "O inventário projetado para o próximo mês é de 1800 toneladas de matéria-prima.",
        "O forecast de produção para o próximo mês é de 4000 computadores.",
        "Foram emitidas 25 ordens de produção este ano.",
        "O inventário de matéria-prima garante a continuidade da produção e evita paradas inesperadas.",
        "Se o forecast for subestimado, a demanda pode não ser atendida, causando perda de vendas e clientes insatisfeitos.",
        "O forecast é uma estimativa da demanda futura, enquanto o pedido real reflete a produção já confirmada.",
        "O maior faturamento mensal deste ano foi de R$ 6 milhões, registrado em julho.",
        "O forecast de vendas orienta o planejamento da produção para garantir que a oferta atenda à demanda esperada.",
        "Um excesso de até 10% no inventário de matéria-prima é considerado aceitável para evitar rupturas na produção.",
        "O faturamento é diretamente proporcional à quantidade vendida, sendo impactado também pelo preço de venda por unidade.",
        "Sim, o forecast foi revisado duas vezes nos últimos três meses devido a mudanças na demanda do mercado.",
        "A previsão de inventário de matéria-prima para o próximo trimestre é de 2500 toneladas.",
        "A baixa no inventário de matéria-prima resultou em uma redução de 15% na produção no último mês.",
        "No ano passado, foram vendidos 18.000 computadores.",
        "A previsão de demanda é feita com base em dados históricos de vendas, análise de mercado e tendências econômicas.",
        "Sim, o faturamento deste trimestre superou o forecast em 12%.",
        "O forecast de vendas do mês passado foi de 2400 unidades.",
        "Foram produzidas 2600 unidades no mês passado.",
        "As vendas do mês passado foram 5% superiores ao forecast.",
        "Foram concluídas 8 ordens de produção no mês passado.",
        "O inventário de matéria-prima foi utilizado em 95%, garantindo a produção contínua no mês passado.",
        "A previsão de crescimento de vendas para o próximo trimestre é de 12%.",
        "A produção planejada para o próximo mês está alinhada ao forecast com uma margem de 3% de ajuste.",
        "Espera-se um impacto positivo de 10% devido à alta demanda no próximo trimestre.",
        "Sim, o forecast de produção do próximo mês foi ajustado para atender à demanda crescente.",
        "A expectativa de crescimento para o faturamento no próximo ano é de 20%.",
        "O desempenho de vendas do mês passado, que superou o forecast em 7%, influenciou a revisão do forecast futuro.",
        "Sim, o inventário de matéria-prima está adequado para atender ao forecast do próximo trimestre."
    ]
}

# Criando o DataFrame
df = pd.DataFrame(data)


In [ ]:
!pip install sentence-transformers
!pip install scikit-learn
!pip install dateparser
!pip install numpy

## ✨ Execute

**SentenceTransformer:** Usado para gerar embeddings de sentenças, que são representações vetoriais das perguntas.

**cosine_similarity:** Calcula a similaridade de cosseno entre dois vetores.

**numpy:** Fornece funções e objetos úteis para manipulações numéricas e matriciais.

**dateparser:** Interpreta e converte expressões de tempo em datas.

**datetime:** Manipula e acessa informações sobre data e hora.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import dateparser
from datetime import datetime

# Função para ajustar expressões temporais na pergunta
def ajustar_temporalidade(pergunta):
    agora = datetime.now()

    # Ajusta "mês passado" e "último mês"
    pergunta = pergunta.replace("mês passado", dateparser.parse("mês passado").strftime("%B de %Y"))
    pergunta = pergunta.replace("último mês", dateparser.parse("mês passado").strftime("%B de %Y"))

    # Ajusta "ano passado" e "último ano"
    pergunta = pergunta.replace("ano passado", dateparser.parse("ano passado").strftime("%Y"))
    pergunta = pergunta.replace("último ano", dateparser.parse("ano passado").strftime("%Y"))

    # Ajusta "último semestre"
    semestre_atual = (agora.month - 1) // 6 + 1
    ultimo_semestre = semestre_atual - 1 if semestre_atual > 1 else 2
    ano_semestre = agora.year if semestre_atual > 1 else agora.year - 1
    semestre_str = f"{ano_semestre} {'Primeiro' if ultimo_semestre == 1 else 'Segundo'} Semestre"
    pergunta = pergunta.replace("último semestre", semestre_str)

    return pergunta

# Função para encontrar a resposta correta com base na pergunta usando sentence-transformers
def encontrar_resposta(transcription, dataframe, model_embeddings):
    # Ajusta a pergunta para lidar com expressões temporais
    transcription = ajustar_temporalidade(transcription)

    # Obtém os embeddings da pergunta do usuário e move para CPU
    pergunta_embedding = model_embeddings.encode(transcription, convert_to_tensor=True).cpu()

    # Inicializa a lista de similaridades
    similaridades = []

    # Calcula a similaridade de cosseno entre a pergunta do usuário e as perguntas do DataFrame
    for i, row in dataframe.iterrows():
        # Ajusta as perguntas do DataFrame para lidar com expressões temporais
        pergunta_df = ajustar_temporalidade(row['Pergunta'])

        # Obtém os embeddings das perguntas do DataFrame e move para CPU
        pergunta_df_embedding = model_embeddings.encode(pergunta_df, convert_to_tensor=True).cpu()
        similaridade = cosine_similarity([pergunta_embedding], [pergunta_df_embedding])
        similaridades.append(similaridade[0][0])

    # Identificar a pergunta mais similar
    melhor_indice = np.argmax(similaridades)
    melhor_pergunta = dataframe.iloc[melhor_indice]['Pergunta']
    melhor_resposta = dataframe.iloc[melhor_indice]['Resposta']

    return melhor_pergunta, melhor_resposta

# Inicializa o modelo de embeddings do sentence-transformers
model_embeddings = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encontrando a resposta mais relevante
pergunta_encontrada, resposta_encontrada = encontrar_resposta(transcription, df, model_embeddings)

print(f"Pergunta encontrada: {pergunta_encontrada}")
print(f"Resposta: {resposta_encontrada}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Pergunta encontrada: Qual foi o maior faturamento mensal deste ano?
Resposta: O maior faturamento mensal deste ano foi de R$ 6 milhões, registrado em julho.


## Integração com API OpenAI 💭

In [ ]:
!pip install openai -q
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


## ✨ Execute

In [ ]:
from openai import OpenAI

#API
client = OpenAI(api_key="Sua_API_KEY")

#Interação com o ChatGPT
chat_completion = client.chat.completions.create(
messages=[
    {"role": "user",
     "content": "melhore a seguinte resposta: " + resposta_encontrada }
    ],
    #modelo recomendado gpt-3.5-turbo
    model="gpt-3.5-turbo",
  )

#Obtém a resposta do chatGPT (array de escolhas, normalmente a primeira resposta é a melhor, por isso índice 0)
chatgpt_response = chat_completion.choices[0].message.content
print(chatgpt_response)

O maior faturamento mensal deste ano, no valor de R$ 6 milhões, foi alcançado em julho.


## 🗣🔊 Sintetizando a resposta do chatGPT com voz 😮💬

In [ ]:
!pip install gTTS

## ✨ Execute

In [ ]:
#Biblioteca google
from gtts import gTTS

#Define o que ele vai ler
gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)
response_audio = "response.wav"

#Salva o áudio
gtts_object.save("response.wav")
display(Audio("response.wav", autoplay=True))